In [40]:
# fetching data from csv file

import pandas as pd

In [41]:
matches = pd.read_csv("matches.csv")


In [42]:
matches.head()

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
1,2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
2,3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
3,4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
4,6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City


In [43]:
matches.shape

(1389, 28)

In [44]:
matches["team"].value_counts()


team
Manchester United           72
West Ham United             72
Newcastle United            72
Brighton and Hove Albion    72
Southampton                 72
Tottenham Hotspur           71
Manchester City             71
Leeds United                71
Wolverhampton Wanderers     71
Arsenal                     71
Crystal Palace              71
Burnley                     71
Chelsea                     70
Aston Villa                 70
Leicester City              70
Everton                     70
West Bromwich Albion        38
Fulham                      38
Liverpool                   38
Sheffield United            38
Brentford                   34
Norwich City                33
Watford                     33
Name: count, dtype: int64

In [45]:

matches[matches["team"] == "Liverpool"]

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
705,1,2020-09-12,17:30,Premier League,Matchweek 1,Sat,Home,W,4.0,3.0,...,Match Report,NaN,20.0,4.0,17.0,0.0,2.0,2.0,2021,Liverpool
706,2,2020-09-20,16:30,Premier League,Matchweek 2,Sun,Away,W,2.0,0.0,...,Match Report,NaN,17.0,5.0,17.7,1.0,0.0,0.0,2021,Liverpool
707,4,2020-09-28,20:00,Premier League,Matchweek 3,Mon,Home,W,3.0,1.0,...,Match Report,NaN,21.0,9.0,16.8,0.0,0.0,0.0,2021,Liverpool
708,6,2020-10-04,19:15,Premier League,Matchweek 4,Sun,Away,L,2.0,7.0,...,Match Report,NaN,14.0,8.0,15.8,1.0,0.0,0.0,2021,Liverpool
709,7,2020-10-17,12:30,Premier League,Matchweek 5,Sat,Away,D,2.0,2.0,...,Match Report,NaN,22.0,8.0,15.0,1.0,0.0,0.0,2021,Liverpool
710,9,2020-10-24,20:00,Premier League,Matchweek 6,Sat,Home,W,2.0,1.0,...,Match Report,NaN,17.0,5.0,18.2,1.0,0.0,0.0,2021,Liverpool
711,11,2020-10-31,17:30,Premier League,Matchweek 7,Sat,Home,W,2.0,1.0,...,Match Report,NaN,8.0,2.0,18.6,1.0,1.0,1.0,2021,Liverpool
712,13,2020-11-08,16:30,Premier League,Matchweek 8,Sun,Away,D,1.0,1.0,...,Match Report,NaN,9.0,2.0,21.5,0.0,1.0,1.0,2021,Liverpool
713,14,2020-11-22,19:15,Premier League,Matchweek 9,Sun,Home,W,3.0,0.0,...,Match Report,NaN,24.0,12.0,11.9,0.0,0.0,0.0,2021,Liverpool
714,16,2020-11-28,12:30,Premier League,Matchweek 10,Sat,Away,D,1.0,1.0,...,Match Report,NaN,6.0,2.0,20.9,0.0,0.0,0.0,2021,Liverpool


In [46]:
matches["round"].value_counts()

round
Matchweek 1     39
Matchweek 2     39
Matchweek 3     39
Matchweek 4     39
Matchweek 5     39
Matchweek 6     39
Matchweek 7     39
Matchweek 8     39
Matchweek 9     39
Matchweek 10    39
Matchweek 11    39
Matchweek 12    39
Matchweek 13    39
Matchweek 14    39
Matchweek 15    39
Matchweek 16    39
Matchweek 17    39
Matchweek 19    39
Matchweek 20    39
Matchweek 25    39
Matchweek 24    39
Matchweek 23    39
Matchweek 34    39
Matchweek 29    39
Matchweek 31    39
Matchweek 26    39
Matchweek 28    39
Matchweek 32    39
Matchweek 18    37
Matchweek 21    37
Matchweek 22    37
Matchweek 27    37
Matchweek 30    37
Matchweek 33    32
Matchweek 35    20
Matchweek 36    20
Matchweek 37    20
Matchweek 38    20
Name: count, dtype: int64

In [47]:
matches.dtypes

Unnamed: 0        int64
date             object
time             object
comp             object
round            object
day              object
venue            object
result           object
gf              float64
ga              float64
opponent         object
xg              float64
xga             float64
poss            float64
attendance      float64
captain          object
formation        object
referee          object
match report     object
notes           float64
sh              float64
sot             float64
dist            float64
fk              float64
pk              float64
pkatt           float64
season            int64
team             object
dtype: object

In [48]:
# ml only works with numerical data or float
matches['date'] = pd.to_datetime(matches['date']) #convert date to datetime format
matches.dtypes

Unnamed: 0               int64
date            datetime64[ns]
time                    object
comp                    object
round                   object
day                     object
venue                   object
result                  object
gf                     float64
ga                     float64
opponent                object
xg                     float64
xga                    float64
poss                   float64
attendance             float64
captain                 object
formation               object
referee                 object
match report            object
notes                  float64
sh                     float64
sot                    float64
dist                   float64
fk                     float64
pk                     float64
pkatt                  float64
season                   int64
team                    object
dtype: object

In [49]:
# predictors 
matches['venue_code'] = matches['venue'].astype('category').cat.codes#from sting to caterogy to numbers

In [50]:
matches["opp_codes"] = matches['opponent'].astype('category').cat.codes


In [51]:
matches

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,sh,sot,dist,fk,pk,pkatt,season,team,venue_code,opp_codes
0,1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,...,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City,0,18
1,2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,...,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City,1,15
2,3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,...,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City,1,0
3,4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,...,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City,0,10
4,6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,...,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City,1,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0.0,4.0,...,8.0,1.0,17.4,0.0,0.0,0.0,2021,Sheffield United,0,18
1385,39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0.0,2.0,...,7.0,0.0,11.4,1.0,0.0,0.0,2021,Sheffield United,1,6
1386,40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1.0,0.0,...,10.0,3.0,17.0,0.0,0.0,0.0,2021,Sheffield United,0,7
1387,41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0.0,1.0,...,11.0,1.0,16.0,1.0,0.0,0.0,2021,Sheffield United,0,14


In [52]:
matches["hour"] = matches['time'].str.replace(":.+", "", regex=True).astype("int")

In [53]:
matches


,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,sot,dist,fk,pk,pkatt,season,team,venue_code,opp_codes,hour
0,1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,...,4.0,16.9,1.0,0.0,0.0,2022,Manchester City,0,18,16
1,2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,...,4.0,17.3,1.0,0.0,0.0,2022,Manchester City,1,15,15
2,3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,...,10.0,14.3,0.0,0.0,0.0,2022,Manchester City,1,0,12
3,4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,...,8.0,14.0,0.0,0.0,0.0,2022,Manchester City,0,10,15
4,6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,...,1.0,15.7,1.0,0.0,0.0,2022,Manchester City,1,17,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0.0,4.0,...,1.0,17.4,0.0,0.0,0.0,2021,Sheffield United,0,18,19
1385,39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0.0,2.0,...,0.0,11.4,1.0,0.0,0.0,2021,Sheffield United,1,6,15
1386,40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1.0,0.0,...,3.0,17.0,0.0,0.0,0.0,2021,Sheffield United,0,7,19
1387,41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0.0,1.0,...,1.0,16.0,1.0,0.0,0.0,2021,Sheffield United,0,14,18


In [54]:
matches["day_code"] = matches['date'].dt.dayofweek

In [55]:
matches


,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,dist,fk,pk,pkatt,season,team,venue_code,opp_codes,hour,day_code
0,1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,...,16.9,1.0,0.0,0.0,2022,Manchester City,0,18,16,6
1,2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,...,17.3,1.0,0.0,0.0,2022,Manchester City,1,15,15,5
2,3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,...,14.3,0.0,0.0,0.0,2022,Manchester City,1,0,12,5
3,4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,...,14.0,0.0,0.0,0.0,2022,Manchester City,0,10,15,5
4,6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,...,15.7,1.0,0.0,0.0,2022,Manchester City,1,17,15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0.0,4.0,...,17.4,0.0,0.0,0.0,2021,Sheffield United,0,18,19,6
1385,39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0.0,2.0,...,11.4,1.0,0.0,0.0,2021,Sheffield United,1,6,15,5
1386,40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1.0,0.0,...,17.0,0.0,0.0,0.0,2021,Sheffield United,0,7,19,6
1387,41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0.0,1.0,...,16.0,1.0,0.0,0.0,2021,Sheffield United,0,14,18,2


In [56]:
matches["target"] = (matches['result'] == "W").astype("int")

In [57]:
matches

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,fk,pk,pkatt,season,team,venue_code,opp_codes,hour,day_code,target
0,1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0.0,1.0,...,1.0,0.0,0.0,2022,Manchester City,0,18,16,6,0
1,2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5.0,0.0,...,1.0,0.0,0.0,2022,Manchester City,1,15,15,5,1
2,3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5.0,0.0,...,0.0,0.0,0.0,2022,Manchester City,1,0,12,5,1
3,4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1.0,0.0,...,0.0,0.0,0.0,2022,Manchester City,0,10,15,5,1
4,6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0.0,0.0,...,1.0,0.0,0.0,2022,Manchester City,1,17,15,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384,38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0.0,4.0,...,0.0,0.0,0.0,2021,Sheffield United,0,18,19,6,0
1385,39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0.0,2.0,...,1.0,0.0,0.0,2021,Sheffield United,1,6,15,5,0
1386,40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1.0,0.0,...,0.0,0.0,0.0,2021,Sheffield United,0,7,19,6,1
1387,41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0.0,1.0,...,1.0,0.0,0.0,2021,Sheffield United,0,14,18,2,0


In [58]:
# creating model
from sklearn.ensemble import RandomForestClassifier #Random Forest model can pickup non linear relationships


In [59]:
# random forest model is a series of decision trees the more the estimator the better the model generalizes the more accurate but the longer it takes to train
# min samples is the minimum number of samples required to split an internal node the higher the less overfitting but the less accurate
#  random state is for reproducibility 
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [60]:
train = matches[matches['date'] < '2022-01-01']
test = matches[matches['date'] >= '2022-01-01']

In [61]:
predictors = ['venue_code', 'opp_codes', 'hour', 'day_code']

In [62]:
rf.fit(train[predictors], train['target'])

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",50
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",10
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [63]:
preds = rf.predict(test[predictors])

In [64]:
from sklearn.metrics import accuracy_score, precision_score

In [65]:
acc = accuracy_score(test['target'], preds)

In [66]:
acc

0.6134751773049646

In [67]:
combined = pd.DataFrame(dict(actual=test['target'], prediction=preds))

In [68]:
pd.crosstab(index=combined["actual"], columns=combined["prediction"])

prediction,0,1
actual,,
0,144,31
1,78,29


In [69]:
precision_score(test['target'], preds)


0.48333333333333334

In [70]:
# creates a dataframe for each team showing their matches   
grouped_matches = matches.groupby("team")

In [71]:
group = grouped_matches.get_group("Manchester United")

In [72]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)

    return group
    

In [73]:
cols = ['gf', 'ga', 'sh', 'sot', 'dist', 'fk', 'pk', 'pkatt']
new_cols = [f'{c}_rolling' for c in cols]



In [74]:
new_cols

['gf_rolling',
 'ga_rolling',
 'sh_rolling',
 'sot_rolling',
 'dist_rolling',
 'fk_rolling',
 'pk_rolling',
 'pkatt_rolling']

In [75]:
rolling_averages(group, cols, new_cols)

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
670,5,2020-10-17,20:00,Premier League,Matchweek 5,Sat,Away,W,4.0,1.0,...,5,1,1.666667,3.666667,9.000000,2.333333,19.800000,0.333333,0.666667,0.666667
671,7,2020-10-24,17:30,Premier League,Matchweek 6,Sat,Home,D,0.0,0.0,...,5,0,2.666667,3.000000,12.333333,4.666667,20.000000,0.000000,0.666667,1.000000
672,9,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Home,L,0.0,1.0,...,6,0,1.666667,2.333333,15.000000,5.333333,19.700000,0.000000,0.333333,0.666667
673,11,2020-11-07,12:30,Premier League,Matchweek 8,Sat,Away,W,3.0,1.0,...,5,1,1.333333,0.666667,16.666667,5.666667,18.300000,0.000000,0.000000,0.333333
674,12,2020-11-21,20:00,Premier League,Matchweek 9,Sat,Home,W,1.0,0.0,...,5,1,1.000000,0.666667,12.000000,3.666667,18.733333,0.666667,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,40,2022-04-02,17:30,Premier League,Matchweek 31,Sat,Home,D,1.0,1.0,...,5,0,1.333333,2.000000,12.666667,3.666667,15.333333,0.333333,0.000000,0.000000
161,41,2022-04-09,12:30,Premier League,Matchweek 32,Sat,Away,L,0.0,1.0,...,5,0,1.666667,2.333333,9.000000,4.333333,14.333333,0.000000,0.000000,0.000000
162,42,2022-04-16,15:00,Premier League,Matchweek 33,Sat,Home,W,3.0,2.0,...,5,1,1.333333,1.333333,11.333333,5.000000,15.333333,0.000000,0.000000,0.000000
163,43,2022-04-19,20:00,Premier League,Matchweek 30,Tue,Away,L,0.0,4.0,...,1,0,1.333333,1.333333,14.333333,6.000000,15.666667,0.333333,0.000000,0.000000


In [76]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

C:\Users\matzi\AppData\Local\Temp\ipykernel_13128\4052147919.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [77]:
matches_rolling

Unnamed: 0       date   time            comp  \
team                                                                        
Arsenal                 898           6 2020-10-04  14:00  Premier League   
                        899           7 2020-10-17  17:30  Premier League   
                        900           9 2020-10-25  19:15  Premier League   
                        901          11 2020-11-01  16:30  Premier League   
                        902          13 2020-11-08  19:15  Premier League   
...                                 ...        ...    ...             ...   
Wolverhampton Wanderers 227          32 2022-03-13  14:00  Premier League   
                        228          33 2022-03-18  20:00  Premier League   
                        229          34 2022-04-02  15:00  Premier League   
                        230          35 2022-04-08  20:00  Premier League   
                        231          36 2022-04-24  14:00  Premier League   

                                    round  day venue result   gf   ga  ...  \
team                                                                   ...   
Arsenal                 898   Matchweek 4  Sun  Home      W  2.0  1.0  ...   
                        899   Matchweek 5  Sat  Away      L  0.0  1.0  ...   
                        900   Matchweek 6  Sun  Home      L  0.0  1.0  ...   
                        901   Matchweek 7  Sun  Away      W  1.0  0.0  ...   
                        902   Matchweek 8  Sun  Home      L  0.0  3.0  ...   
...                                   ...  ...   ...    ...  ...  ...  ...   
Wolverhampton Wanderers 227  Matchweek 29  Sun  Away      W  1.0  0.0  ...   
                        228  Matchweek 30  Fri  Home      L  2.0  3.0  ...   
                        229  Matchweek 31  Sat  Home      W  2.0  1.0  ...   
                        230  Matchweek 32  Fri  Away      L  0.0  1.0  ...   
                        231  Matchweek 34  Sun  Away      L  0.0  1.0  ...   

                            day_code  target  gf_rolling  ga_rolling  \
team                                                                   
Arsenal                 898        6       1    2.000000    1.333333   
                        899        5       0    1.666667    1.666667   
                        900        6       0    1.000000    1.666667   
                        901        6       1    0.666667    1.000000   
                        902        6       0    0.333333    0.666667   
...                              ...     ...         ...         ...   
Wolverhampton Wanderers 227        6       1    1.333333    1.000000   
                        228        4       0    1.666667    0.666667   
                        229        5       1    2.333333    1.000000   
                        230        4       0    1.666667    1.333333   
                        231        6       0    1.333333    1.666667   

                             sh_rolling sot_rolling dist_rolling fk_rolling  \
team                                                                          
Arsenal                 898    7.666667    3.666667    14.733333   0.666667   
                        899    5.333333    3.666667    15.766667   0.000000   
                        900    7.000000    3.666667    16.733333   0.666667   
                        901    9.666667    4.000000    16.033333   1.000000   
                        902    9.666667    2.666667    18.033333   1.000000   
...                                 ...         ...          ...        ...   
Wolverhampton Wanderers 227   12.333333    3.666667    19.300000   0.000000   
                        228   12.333333    4.333333    19.600000   0.000000   
                        229   13.000000    5.333333    19.833333   0.000000   
                        230   13.000000    5.000000    18.533333   0.000000   
                        231   10.000000    4.666667    17.633333   0.000000   

                            pk_rolling  pkatt_r

In [78]:
matches_rolling = matches_rolling.droplevel("team")

In [79]:
matches_rolling

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
898,6,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,...,6,1,2.000000,1.333333,7.666667,3.666667,14.733333,0.666667,0.000000,0.000000
899,7,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,...,5,0,1.666667,1.666667,5.333333,3.666667,15.766667,0.000000,0.000000,0.000000
900,9,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,...,6,0,1.000000,1.666667,7.000000,3.666667,16.733333,0.666667,0.000000,0.000000
901,11,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,...,6,1,0.666667,1.000000,9.666667,4.000000,16.033333,1.000000,0.000000,0.000000
902,13,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,...,6,0,0.333333,0.666667,9.666667,2.666667,18.033333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,32,2022-03-13,14:00,Premier League,Matchweek 29,Sun,Away,W,1.0,0.0,...,6,1,1.333333,1.000000,12.333333,3.666667,19.300000,0.000000,0.000000,0.000000
228,33,2022-03-18,20:00,Premier League,Matchweek 30,Fri,Home,L,2.0,3.0,...,4,0,1.666667,0.666667,12.333333,4.333333,19.600000,0.000000,0.000000,0.000000
229,34,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Home,W,2.0,1.0,...,5,1,2.333333,1.000000,13.000000,5.333333,19.833333,0.000000,0.000000,0.000000
230,35,2022-04-08,20:00,Premier League,Matchweek 32,Fri,Away,L,0.0,1.0,...,4,0,1.666667,1.333333,13.000000,5.000000,18.533333,0.000000,0.000000,0.000000


In [80]:
matches_rolling.index = range(matches_rolling.shape[0])

In [81]:
matches_rolling

,Unnamed: 0,date,time,comp,round,day,venue,result,gf,ga,...,day_code,target,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,6,2020-10-04,14:00,Premier League,Matchweek 4,Sun,Home,W,2.0,1.0,...,6,1,2.000000,1.333333,7.666667,3.666667,14.733333,0.666667,0.000000,0.000000
1,7,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Away,L,0.0,1.0,...,5,0,1.666667,1.666667,5.333333,3.666667,15.766667,0.000000,0.000000,0.000000
2,9,2020-10-25,19:15,Premier League,Matchweek 6,Sun,Home,L,0.0,1.0,...,6,0,1.000000,1.666667,7.000000,3.666667,16.733333,0.666667,0.000000,0.000000
3,11,2020-11-01,16:30,Premier League,Matchweek 7,Sun,Away,W,1.0,0.0,...,6,1,0.666667,1.000000,9.666667,4.000000,16.033333,1.000000,0.000000,0.000000
4,13,2020-11-08,19:15,Premier League,Matchweek 8,Sun,Home,L,0.0,3.0,...,6,0,0.333333,0.666667,9.666667,2.666667,18.033333,1.000000,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1312,32,2022-03-13,14:00,Premier League,Matchweek 29,Sun,Away,W,1.0,0.0,...,6,1,1.333333,1.000000,12.333333,3.666667,19.300000,0.000000,0.000000,0.000000
1313,33,2022-03-18,20:00,Premier League,Matchweek 30,Fri,Home,L,2.0,3.0,...,4,0,1.666667,0.666667,12.333333,4.333333,19.600000,0.000000,0.000000,0.000000
1314,34,2022-04-02,15:00,Premier League,Matchweek 31,Sat,Home,W,2.0,1.0,...,5,1,2.333333,1.000000,13.000000,5.333333,19.833333,0.000000,0.000000,0.000000
1315,35,2022-04-08,20:00,Premier League,Matchweek 32,Fri,Away,L,0.0,1.0,...,4,0,1.666667,1.333333,13.000000,5.000000,18.533333,0.000000,0.000000,0.000000


In [82]:
# retraining the model for rolling averages features
def make_predictions(data, predictors, model=rf):
    train = data[data['date'] < '2022-01-01']
    test = data[data['date'] >= '2022-01-01']

    rf.fit(train[predictors], train['target'])

    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test['target'], prediction=preds), index=test.index)
    precision = precision_score(test['target'], preds)
    return combined, precision

In [83]:
combined, precision = make_predictions(matches_rolling, predictors + new_cols)  


In [84]:
precision

0.625

In [85]:
combined

,actual,prediction
54,0,0
55,0,0
56,1,0
57,1,0
58,1,1
...,...,...
1312,1,0
1313,0,0
1314,1,0
1315,0,0


In [86]:
combined = combined.merge(matches_rolling[["team", "opponent", "date", "result"]], left_index=True, right_index=True)

In [87]:
combined

,actual,prediction,team,opponent,date,result
54,0,0,Arsenal,Manchester City,2022-01-01,L
55,0,0,Arsenal,Burnley,2022-01-23,D
56,1,0,Arsenal,Wolves,2022-02-10,W
57,1,0,Arsenal,Brentford,2022-02-19,W
58,1,1,Arsenal,Wolves,2022-02-24,W
...,...,...,...,...,...,...
1312,1,0,Wolverhampton Wanderers,Everton,2022-03-13,W
1313,0,0,Wolverhampton Wanderers,Leeds United,2022-03-18,L
1314,1,0,Wolverhampton Wanderers,Aston Villa,2022-04-02,W
1315,0,0,Wolverhampton Wanderers,Newcastle Utd,2022-04-08,L


In [88]:
# combining home and away predictions
class MissingDict(dict):
    __missing__ = lambda self, key: key

maps_values = {
    "Brighton and Hove Albion": "Brighton",
    "Manchester United": "Manchester Utd",
    "Newcastle United": "Newcaste Utd",
    "Tottenham Hotspur": "Tottenham",
    "West Ham United": "West Ham",
    "Wolverhampton Wanderers": "Wolves"
}

mapping = MissingDict(**maps_values)

In [89]:
mapping["Arsenal"]


'Arsenal'

In [90]:
mapping["West Ham United"]

'West Ham'

In [92]:
combined["new_teams"] = combined["team"].map(mapping)

In [93]:
combined 

,actual,prediction,team,opponent,date,result,new_teams
54,0,0,Arsenal,Manchester City,2022-01-01,L,Arsenal
55,0,0,Arsenal,Burnley,2022-01-23,D,Arsenal
56,1,0,Arsenal,Wolves,2022-02-10,W,Arsenal
57,1,0,Arsenal,Brentford,2022-02-19,W,Arsenal
58,1,1,Arsenal,Wolves,2022-02-24,W,Arsenal
...,...,...,...,...,...,...,...
1312,1,0,Wolverhampton Wanderers,Everton,2022-03-13,W,Wolves
1313,0,0,Wolverhampton Wanderers,Leeds United,2022-03-18,L,Wolves
1314,1,0,Wolverhampton Wanderers,Aston Villa,2022-04-02,W,Wolves
1315,0,0,Wolverhampton Wanderers,Newcastle Utd,2022-04-08,L,Wolves


In [94]:
merged = combined.merge(combined, left_on=["date", "new_teams"], right_on=["date", "opponent"])
# look for new team field and merge with opponent predictions 

In [96]:
merged[(merged["prediction_x"] == 1) & (merged["prediction_y"] == 0)]["actual_x"].value_counts()


actual_x
1    23
0    11
Name: count, dtype: int64

In [97]:
23/40


0.575